In [1]:
import os
import pandas as pd

from portfolio_optimizer.constants import DATA_DIRECTORY, RAW_DATA_DIRECTORY
from portfolio_optimizer.utils.return_metrics import get_returns, get_mean_returns, get_portfolio_return
from portfolio_optimizer.utils.risk_metrics import get_portfolio_std
from portfolio_optimizer.optimization.mean_variance_optimizer import MeanVariancePortfolioOptimizer, mean_variance_optimization

In [2]:
# Read the StockPrices.csv file into a DataFrame called stocks
file_path = os.path.join(RAW_DATA_DIRECTORY, 'StockPrices.csv')
stocks = pd.read_csv(file_path, index_col=0, parse_dates=True)
stocks.head()

/tmp/ipykernel_13981/2196807991.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stocks = pd.read_csv(file_path, index_col=0, parse_dates=True)


,HDFCBANK,HINDUNILVR,IOC,SUNPHARMA,TATAMOTORS
Feb-25,1732.400024,2190.250000,113.489998,1593.250000,620.650024
Jan-25,1698.750000,2468.800049,128.490005,1733.502686,716.099976
Dec-24,1772.849976,2326.850098,136.410004,1875.049683,740.150024
Nov-24,1796.050049,2496.149902,138.630005,1770.231445,786.450012
Oct-24,1735.699951,2518.222656,142.619995,1837.824097,834.049988


In [3]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, Feb-25 to Feb-20
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   HDFCBANK    61 non-null     float64
 1   HINDUNILVR  61 non-null     float64
 2   IOC         61 non-null     float64
 3   SUNPHARMA   61 non-null     float64
 4   TATAMOTORS  61 non-null     float64
dtypes: float64(5)
memory usage: 2.9+ KB


In [4]:
stocks.describe()

,HDFCBANK,HINDUNILVR,IOC,SUNPHARMA,TATAMOTORS
count,61.000000,61.000000,61.000000,61.000000,61.000000
mean,1441.388793,2329.406014,82.765374,983.603603,496.174673
std,219.048632,232.979216,42.663505,435.962623,282.850631
min,827.405640,1907.843018,34.885338,335.035889,70.611343
25%,1375.287964,2152.160889,54.110897,642.327576,299.936707
50%,1455.183594,2326.850098,67.738190,916.104858,446.824219
75%,1569.864014,2502.819336,107.265648,1207.769775,642.317871
max,1796.050049,2946.566895,181.669998,1915.157959,1156.650024


In [5]:
# Convert the index to date object
stocks.index = pd.to_datetime(stocks.index, format='%b-%y')

In [6]:
stocks.head()

,HDFCBANK,HINDUNILVR,IOC,SUNPHARMA,TATAMOTORS
2025-02-01,1732.400024,2190.250000,113.489998,1593.250000,620.650024
2025-01-01,1698.750000,2468.800049,128.490005,1733.502686,716.099976
2024-12-01,1772.849976,2326.850098,136.410004,1875.049683,740.150024
2024-11-01,1796.050049,2496.149902,138.630005,1770.231445,786.450012
2024-10-01,1735.699951,2518.222656,142.619995,1837.824097,834.049988


In [7]:
# Get the stocks log returns
returns = get_returns(stocks)
returns

,HDFCBANK,HINDUNILVR,IOC,SUNPHARMA,TATAMOTORS
2020-03-01,-0.268119,0.056903,-0.189894,-0.055243,-0.449011
2020-04-01,0.162316,-0.045029,0.031231,0.318337,0.312456
2020-05-01,-0.050060,-0.062711,-0.011876,0.021100,-0.067024
2020-06-01,0.120002,0.066847,0.025841,-0.002741,0.129310
2020-07-01,-0.031008,0.018050,0.036321,0.124220,0.065140
2020-08-01,0.080413,-0.041880,-0.029960,-0.019671,0.368371
2020-09-01,-0.033383,-0.023189,-0.138695,-0.038059,-0.069134
2020-10-01,0.097302,0.007958,0.076455,-0.069338,-0.004876
2020-11-01,0.217397,0.032299,0.063482,0.098551,0.359593
2020-12-01,-0.003158,0.120288,0.075059,0.157725,0.019407


In [8]:
# Get the mean returns
mean_returns = get_mean_returns(returns)
mean_returns

HDFCBANK      0.009748
HINDUNILVR    0.003499
IOC           0.018411
SUNPHARMA     0.028001
TATAMOTORS    0.037586
dtype: float64

In [9]:
# Get the covariance matrix
cov_matrix = returns.cov()
cov_matrix

,HDFCBANK,HINDUNILVR,IOC,SUNPHARMA,TATAMOTORS
HDFCBANK,0.005177,0.000560,0.002237,0.000954,0.005371
HINDUNILVR,0.000560,0.004403,0.001219,0.000665,-0.000680
IOC,0.002237,0.001219,0.007942,0.002581,0.006984
SUNPHARMA,0.000954,0.000665,0.002581,0.005752,0.003387
TATAMOTORS,0.005371,-0.000680,0.006984,0.003387,0.022767


In [10]:
# Get the portfolio return
portfolio_return = get_portfolio_return(mean_returns)
portfolio_return

np.float64(0.019449099291898808)

In [11]:
get_portfolio_std(covariance_matrix=cov_matrix, returns=mean_returns)

array([[0.0608591]])

In [12]:

# # Example usage:
# optimizer = PortfolioOptimizer(mu=mean_returns, cov_matrix=cov_matrix)

# # Minimize volatility with long-only constraint
# weights = optimizer.optimize('min_volatility', ['long_only'], {})

# # Maximize return with required return constraint
# weights = optimizer.optimize('max_return', ['required_return'], {'required_return': 0.03})

# # Minimize volatility with max leverage constraint
# weights = optimizer.optimize('min_volatility', ['max_leverage'], {'max_leverage': 1.5})

In [13]:
best_weights = mean_variance_optimization(mu=mean_returns, cov_matrix=cov_matrix)

In [14]:
best_weights.round(4)*100 # In percentages

array([30.63, 39.75,  3.82, 25.8 ,  0.  ])

In [15]:
# Best Portfolio Return
get_portfolio_return(mean_returns, best_weights)

np.float64(0.01230578801002344)

In [16]:
# Best Portfolio Standard Deviation
get_portfolio_std(covariance_matrix=cov_matrix, returns=mean_returns, weights=best_weights)

array([[0.04625593]])

In [19]:
constraints = ['long_only']
optimizer = MeanVariancePortfolioOptimizer(mu=mean_returns.values, cov_matrix=cov_matrix.values)
objective = 'sharpe_ratio'
params = {'risk_free_rate': 0.05}
weights = optimizer.optimize(objective, constraints, params=params)
weights.round(4)*100 # In percentages

array([ 0.  , -0.  ,  0.  , 45.04, 54.96])

In [20]:
get_portfolio_std(covariance_matrix=cov_matrix, returns=mean_returns, weights=weights)*100

array([[9.85884321]])

In [21]:
get_portfolio_return(mean_returns, weights)*100

np.float64(3.3268470337525042)